<a href="https://colab.research.google.com/github/Yonas-Berhe/machine-learning-zoomcamp-homework/blob/main/ML_08_DeepLearnig_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ML 8 Homework**

In [ ]:
!pip install wget

In [23]:
# import libraries

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wget
import torchvision.models as models
from torchvision import transforms

**Reproducibility**

In [7]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Data **Preparation**

In [14]:
filename = wget.download('https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip')


In [15]:
import zipfile
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref. extractall('extracted_data')
print('Extraction complete!')

Extraction complete!


**Model**

In [22]:
import os
from torch.utils.data import Dataset

class ClothingDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [24]:
input_size = 224

# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),           # Rotate up to 10 degrees
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),  # Zoom
    transforms.RandomHorizontalFlip(),       # Horizontal flip
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [45]:
from torch.utils.data import DataLoader

train_dataset = ClothingDataset(
    data_dir='/content/extracted_data/data/train',
    transform=train_transforms
)

validation_dataset = ClothingDataset(
    data_dir='/content/extracted_data/data/test',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [33]:
def train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device):
    best_val_accuracy = 0.0  # Initialize variable to track the best validation accuracy

In [36]:
class BinaryClassificationCNN(nn.Module):
    def __init__(self):
        super(BinaryClassificationCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        self.relu = nn. ReLU()
        self. maxpool = nn.MaxPool2d(kernel_size=(2, 2))
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

        # REMOVED: self.sigmoid (BCEWithLogitsLoss applies it internally)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self. fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # Output raw logits (no sigmoid)
        return x


def make_model():
    """
    Initialize and return the CNN model with optimizer and loss function.

    Returns:
        tuple: (model, optimizer, criterion)
    """
    model = BinaryClassificationCNN()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

    # Use BCEWithLogitsLoss for better numerical stability
    criterion = nn.BCEWithLogitsLoss()

    print(model)
    print(f"\nTotal parameters: {sum(p. numel() for p in model. parameters())}")

    return model, optimizer

**Question 1**

In [37]:
criterion = nn.BCEWithLogitsLoss()

*nn.BCEWithLogitsLoss()*

* Expects raw logits (unbounded values) as input
* Combines sigmoid + BCELoss in one operation
* More numerically stable (avoids numerical errors from separate sigmoid + log operations)
* Formula: Applies sigmoid internally, then computes BCE

In [38]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BinaryClassificationCNN()
model.to(device);

In [39]:
num_epochs = 2

optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

model, optimizer = make_model()


train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)

BinaryClassificationCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

Total parameters: 20073473


**Question 2**

In [40]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


**Generators and Training**

In [49]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

In [51]:
from PIL import Image

In [52]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6773, Acc: 0.6242, Val Loss: 0.6895, Val Acc: 0.5771
Epoch 2/10, Loss: 0.6788, Acc: 0.6005, Val Loss: 0.6461, Val Acc: 0.6766
Epoch 3/10, Loss: 0.6438, Acc: 0.6779, Val Loss: 0.6590, Val Acc: 0.6617
Epoch 4/10, Loss: 0.6108, Acc: 0.6979, Val Loss: 0.6802, Val Acc: 0.6219
Epoch 5/10, Loss: 0.6043, Acc: 0.7016, Val Loss: 0.6429, Val Acc: 0.6567
Epoch 6/10, Loss: 0.5757, Acc: 0.7278, Val Loss: 0.6626, Val Acc: 0.6368
Epoch 7/10, Loss: 0.5419, Acc: 0.7653, Val Loss: 0.6428, Val Acc: 0.6617
Epoch 8/10, Loss: 0.5648, Acc: 0.7316, Val Loss: 0.6760, Val Acc: 0.6617
Epoch 9/10, Loss: 0.5091, Acc: 0.7878, Val Loss: 0.6481, Val Acc: 0.6517
Epoch 10/10, Loss: 0.5464, Acc: 0.7278, Val Loss: 0.6771, Val Acc: 0.6468


In [ ]:
# Epoch 1/10, Loss: 0.6773, Acc: 0.6242, Val Loss: 0.6895, Val Acc: 0.5771
# Epoch 2/10, Loss: 0.6788, Acc: 0.6005, Val Loss: 0.6461, Val Acc: 0.6766
# Epoch 3/10, Loss: 0.6438, Acc: 0.6779, Val Loss: 0.6590, Val Acc: 0.6617
# Epoch 4/10, Loss: 0.6108, Acc: 0.6979, Val Loss: 0.6802, Val Acc: 0.6219
# Epoch 5/10, Loss: 0.6043, Acc: 0.7016, Val Loss: 0.6429, Val Acc: 0.6567
# Epoch 6/10, Loss: 0.5757, Acc: 0.7278, Val Loss: 0.6626, Val Acc: 0.6368
# Epoch 7/10, Loss: 0.5419, Acc: 0.7653, Val Loss: 0.6428, Val Acc: 0.6617
# Epoch 8/10, Loss: 0.5648, Acc: 0.7316, Val Loss: 0.6760, Val Acc: 0.6617
# Epoch 9/10, Loss: 0.5091, Acc: 0.7878, Val Loss: 0.6481, Val Acc: 0.6517
# Epoch 10/10, Loss: 0.5464, Acc: 0.7278, Val Loss: 0.6771, Val Acc: 0.6468

**Question 3**

In [53]:
import numpy as np

train_accuracies = history['acc']
median_accuracy = np.median(train_accuracies)

print(f"Median training accuracy: {median_accuracy:.4f}")

Median training accuracy: 0.7147


**Question 4**

In [55]:
import numpy as np

train_losses = history['loss']
std_dev_loss = np.std(train_losses)

print(f"Standard deviation of training loss: {std_dev_loss:.4f}")

Standard deviation of training loss: 0.0551


**Data Augmentation**


In [56]:
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),

(RandomHorizontalFlip(p=0.5),)

Question 5

In [57]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.5297, Acc: 0.7441, Val Loss: 1.4168, Val Acc: 0.5920
Epoch 2/10, Loss: 0.6766, Acc: 0.5980, Val Loss: 0.7623, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5582, Acc: 0.7303, Val Loss: 0.6238, Val Acc: 0.6617
Epoch 4/10, Loss: 0.5018, Acc: 0.7728, Val Loss: 0.7846, Val Acc: 0.6169
Epoch 5/10, Loss: 0.4738, Acc: 0.7890, Val Loss: 0.6643, Val Acc: 0.6716
Epoch 6/10, Loss: 0.4610, Acc: 0.8027, Val Loss: 0.6651, Val Acc: 0.6468
Epoch 7/10, Loss: 0.4430, Acc: 0.8115, Val Loss: 0.6500, Val Acc: 0.6766
Epoch 8/10, Loss: 0.4476, Acc: 0.8090, Val Loss: 0.7675, Val Acc: 0.6567
Epoch 9/10, Loss: 0.4289, Acc: 0.8240, Val Loss: 2.0070, Val Acc: 0.6119
Epoch 10/10, Loss: 0.7877, Acc: 0.5418, Val Loss: 0.6665, Val Acc: 0.5821


In [ ]:
# Epoch 1/10, Loss: 0.5297, Acc: 0.7441, Val Loss: 1.4168, Val Acc: 0.5920
# Epoch 2/10, Loss: 0.6766, Acc: 0.5980, Val Loss: 0.7623, Val Acc: 0.6318
# Epoch 3/10, Loss: 0.5582, Acc: 0.7303, Val Loss: 0.6238, Val Acc: 0.6617
# Epoch 4/10, Loss: 0.5018, Acc: 0.7728, Val Loss: 0.7846, Val Acc: 0.6169
# Epoch 5/10, Loss: 0.4738, Acc: 0.7890, Val Loss: 0.6643, Val Acc: 0.6716
# Epoch 6/10, Loss: 0.4610, Acc: 0.8027, Val Loss: 0.6651, Val Acc: 0.6468
# Epoch 7/10, Loss: 0.4430, Acc: 0.8115, Val Loss: 0.6500, Val Acc: 0.6766
# Epoch 8/10, Loss: 0.4476, Acc: 0.8090, Val Loss: 0.7675, Val Acc: 0.6567
# Epoch 9/10, Loss: 0.4289, Acc: 0.8240, Val Loss: 2.0070, Val Acc: 0.6119
# Epoch 10/10, Loss: 0.7877, Acc: 0.5418, Val Loss: 0.6665, Val Acc: 0.5821

**Question 5**

In [59]:
import numpy as np

test_losses = history['val_loss']
mean_test_loss = np.mean(test_losses)

print(f"Mean test loss accuracy: {mean_test_loss:.4f}")

Mean test loss accuracy: 0.9008


**Question 6**

In [60]:
import numpy as np

val_accuracies = history['val_acc']
# Get the last 5 epochs' accuracies
last_5_val_accuracies = val_accuracies[-5:]

mean_last_5_val_acc = np.mean(last_5_val_accuracies)

print(f"Average test accuracy for the last 5 epochs: {mean_last_5_val_acc:.4f}")

Average test accuracy for the last 5 epochs: 0.6348
